Data can be download from the following link
https://search.earthdata.nasa.gov/portal/ghrc/search?fsm0=Atmospheric%20Electricity&fst0=Atmosphere

* Last modified 7-19-2022

In [1]:
import re
import pandas as pd
from datetime import datetime

In [2]:
fname = "NALMA_220712_235000_0600.dat"

In [3]:
# Match string
expr_data ='([^+-]\d+.\d+)'\
    '(\s+[+-]?\d+[.]?\d+)'\
        '(\s+[+-]?\d+[.]?\d+)'\
            '(\s+[+-]?\d+[.]?\d+)'\
                '(\s+[+-]?\d+[.]?\d+)'\
                    '(\s+[+-]?\d+[.]?\d+)'\
                        '(\s[0-9a-zA-Z]*)'

expr_station_info = '(?<=(?:Sta_info):[\s*])'\
    '([A-Z]?)(\D+)([+-]?\d+[.]?\d+)'\
        '(\s+[+-]\d+[.]\d+)'\
            '(\s+\d+[.]?\d+)'\
                '(\s+\d+[.]?\d+)'\
                    '(\s+\d+)(\s+\d+)'
expr_station_data = '(?<=(?:Sta_data):[\s*])'\
    '([A-Z]?)(\D+)'\
        '(\d+)(\s+\d+)'\
            '(\s+\d+)'\
                '(\s+\d+)'\
                    '(\s+\d+[.]?\d+)'\
                        '(\s+\d+[.]?\d+)'\
                            '(\s+\w+)'

expr = [expr_data,expr_station_info,expr_station_data]

In [4]:

def extract_NALMA(expr,fname):

    with open(fname,mode='rt') as f:
        nalma_data = f.read()

        array_data = nalma_data[0:709].split('\n')
        array_data = array_data[2:]
        nalm_arry_info = pd.DataFrame([text.split(': ') for text in array_data],columns=['Features','Values']).astype('string')

        dates = datetime.strptime(nalm_arry_info.loc[2,'Values'],'%d/%m/%y %H:%M:%S').date()




        get_data = re.findall(expr[0], nalma_data)
        df = pd.DataFrame(get_data,columns=['Time (UT sec of day)','Latitude','Longitude', 'Altitude (m)', 'reduced chi^2', 'P(dBW)','mask'])
        df = df.astype({'Time (UT sec of day)':'float32',
                        'Latitude':'float32',
                        'Longitude':'float32',
                        'Altitude (m)':'float32',
                        'reduced chi^2':'float32',
                        'P(dBW)':'float32',
                        'mask':'object'});

        df['date'] = dates

        station_info = re.findall(expr[1],nalma_data)
        stinfo_ = pd.DataFrame(station_info)

        station_data = re.findall(expr[2],nalma_data)
        stdata_ = pd.DataFrame(station_data)

        df_stinfo = pd.concat([stinfo_,stdata_],axis=1,join='outer',ignore_index=True).drop(columns=[8,9])

        df_stinfo = df_stinfo.rename(columns={0:'id',1:'name',2:'lat(d)',
        3:'lon(d)',4:'alt(m)',5:'delay(ns)',
        6:'board_rev',7:'rec_ch',10:'win(us)',
        11:'dec_win(us)',12:'data_ver',13:'rms_error(ns)',
        14:'sources(%)',15:'<P/P_m>',16:'active'})

        df_stinfo[['name','id']] = df_stinfo[['name','id']].astype('string')
        df_stinfo[df_stinfo.columns[2:-1].to_list()] = df_stinfo[df_stinfo.columns[2:-1].to_list()].astype('float')

        return df,df_stinfo,nalm_arry_info
        

In [5]:
df,df_stinfo,arry_info = extract_NALMA(expr,fname)

In [6]:
df

,Time (UT sec of day),Latitude,Longitude,Altitude (m),reduced chi^2,P(dBW),mask,date
0,85800.070312,35.747196,-86.212585,9738.049805,0.00,5.1,0x009f,2022-12-07
1,85800.093750,35.715305,-86.261909,16402.820312,2.31,-0.8,0x00bf,2022-12-07
2,85800.109375,35.738178,-86.221741,9756.089844,0.20,2.0,0x009f,2022-12-07
3,85800.109375,35.752926,-86.243515,24828.080078,3.53,2.7,0x003f,2022-12-07
4,85800.117188,35.746391,-86.207420,9738.559570,0.44,1.2,0x009f,2022-12-07
...,...,...,...,...,...,...,...,...
112500,86399.953125,35.933285,-85.762215,13189.690430,0.83,11.4,0x1ac1,2022-12-07
112501,86399.976562,35.562408,-86.206833,11683.780273,2.77,10.6,0x14c6,2022-12-07
112502,86399.976562,35.973660,-86.051941,9976.280273,1.93,5.1,0x08db,2022-12-07
112503,86399.976562,35.985226,-86.050087,17343.919922,2.73,9.3,0x1a8f,2022-12-07


In [7]:
df_stinfo

,id,name,lat(d),lon(d),alt(m),delay(ns),board_rev,rec_ch,win(us),dec_win(us),data_ver,rms_error(ns),sources(%),<P/P_m>,active
0,A,firetower,34.809259,-87.035723,207.62,720.0,3.0,3.0,80.0,10.0,70.0,62622.0,55.7,0.19,A
1,B,boeing,34.643381,-86.771402,174.40,822.0,3.0,3.0,80.0,10.0,70.0,94928.0,84.4,0.08,A
2,C,annex,34.725354,-86.644978,198.30,815.0,3.0,3.0,80.0,12.0,70.0,30618.0,27.2,1.33,A
3,D,keel,34.665633,-86.358613,486.50,999.0,3.0,3.0,80.0,10.0,70.0,101487.0,90.2,0.80,A
4,E,mtsano,34.745562,-86.512651,507.40,962.0,3.0,3.0,80.0,10.0,70.0,32050.0,28.5,0.37,A
5,F,ardmore,34.983603,-86.839259,257.13,26.0,3.0,3.0,80.0,10.0,70.0,13804.0,12.3,0.31,A
6,G,aamu,34.899694,-86.557849,218.60,945.0,3.0,3.0,80.0,10.0,70.0,38976.0,34.6,1.99,A
7,H,green,34.612191,-86.519687,465.20,954.0,3.0,3.0,80.0,10.0,70.0,99650.0,88.6,0.55,A
8,I,owen,34.786032,-86.824729,229.80,891.0,3.0,3.0,0.0,0.0,70.0,0.0,0.0,0.00,NA
9,J,hospital,34.523138,-86.968164,213.70,918.0,3.0,3.0,80.0,10.0,70.0,66658.0,59.2,5.88,A


In [8]:
arry_info

,Features,Values
0,Analysis program version,10.14.9R
1,File created,Wed Jul 13 03:21:26 2022
2,Data start time,07/12/22 23:50:00
3,Number of seconds analyzed,600
4,Location,NALMA
5,"Coordinate center (lat,lon,alt)",34.7246100 -86.6453300 0.00
6,Coordinate frame,cartesian
7,Maximum diameter of LMA (km),90.299
8,Maximum light-time across LMA (ns),301263
9,Number of stations,14
